In [ ]:
import pandas as pd
import glob
import os


if not os.path.exists('updated_files'):
    os.mkdir('updated_files')

dfs = []
file_list = glob.glob('etfs\*.csv')

#merge all files of etfs
for file in file_list:
    df = pd.read_csv(file)
    sym = os.path.splitext(os.path.basename(file))[0]
    filename = os.path.splitext(os.path.basename(file))
    #print(filename)
    df['Symbol'] = f"{sym}"
    #print(df.head())
    dfs.append(df)
    new_file = f"updated_{filename}"

df1 = pd.concat(dfs, ignore_index=True)

#merge all files of stocks
dfst = []
file_list = glob.glob('stocks\*.csv')

for file in file_list:
    df = pd.read_csv(file)
    sym = os.path.splitext(os.path.basename(file))[0]
    filename = os.path.splitext(os.path.basename(file))
    #print(filename)
    df['Symbol'] = f"{sym}"
    #print(df.head())
    dfst.append(df)
    new_file = f"updated_{filename}"

df2 = pd.concat(dfst, ignore_index=True)
    
# write the merged DataFrame to a new CSV file in the updated_files directory
merged_path = os.path.join('updated_files', 'merged_file.csv')

df3 = pd.read_csv('symbols_valid_meta.csv')

merged_df1 = pd.merge(df1, df3[['Symbol', 'Security Name']], on='Symbol')
merged_df2 = pd.merge(df2, df3[['Symbol', 'Security Name']], on='Symbol')

merged_df= pd.concat([merged_df1, merged_df2], axis=0)
print(len(merged_df1))
print(len(merged_df2))

print(len(merged_df))
merged_df.to_csv(merged_path,index=False)


    

In [8]:
import pyarrow.parquet as pq
import pyarrow as pa
df = pd.read_csv('updated_files\merged_file.csv')
# Convert the 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Set the 'Date' column as the index
df.set_index('Date', inplace=True)

# Calculate the 30-day average of the Trading Volume
df['vol_moving_avg'] = df['Volume'].rolling(window=30).mean()

# Calculate the 30-day Rolling median
df['adj_close_rolling_med'] = df['Adj Close'].rolling(window=30).median()

table = pa.Table.from_pandas(df)
pq.write_table(table, 'feature_engineering.parquet')



In [1]:
print("hello")
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_error, r2_score

print("Hellog")
df = pd.read_parquet('feature_engineering.parquet')
print("yuy")
print(df.head())
# Assume `data` is loaded as a Pandas DataFrame
#df['Date'] = pd.to_datetime(df['Date'])
#df.set_index('Date', inplace=True)

# Remove rows with NaN values
df.dropna(inplace=True)

# Select features and target
features = ['vol_moving_avg', 'adj_close_rolling_med']
target = 'Volume'

X = df[features]
y = df[target]
print(X)
print(y)
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a RandomForestRegressor model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model



model.fit(X_train, y_train)
print("3")



# Make predictions on test data
y_pred = model.predict(X_test)

# Calculate the Mean Absolute Error and Mean Squared Error
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print("success")
print(mae)
print(mse)
print(rmse)
print(r2)

hello
Hellog
yuy
             Open   High    Low  Close  Adj Close    Volume Symbol  \
Date                                                                 
2018-08-15  11.84  11.84  11.74  11.74      11.74   27300.0   AAAU   
2018-08-16  11.78  11.80  11.74  11.74      11.74  428400.0   AAAU   
2018-08-17  11.80  11.82  11.77  11.82      11.82   52400.0   AAAU   
2018-08-20  11.88  11.91  11.85  11.90      11.90   28700.0   AAAU   
2018-08-21  11.92  11.95  11.89  11.93      11.93   30600.0   AAAU   

                           Security Name  vol_moving_avg  \
Date                                                       
2018-08-15  Perth Mint Physical Gold ETF             NaN   
2018-08-16  Perth Mint Physical Gold ETF             NaN   
2018-08-17  Perth Mint Physical Gold ETF             NaN   
2018-08-20  Perth Mint Physical Gold ETF             NaN   
2018-08-21  Perth Mint Physical Gold ETF             NaN   

            adj_close_rolling_med  
Date                               